## Test the pre-trained imported model Road signs


In [ ]:
# useful libraries
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from utils import *

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical

print('Tensorflow version :', tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

%matplotlib inline

### Preprocess the test set

In [ ]:
with open("./traffic-signs-data/test.p", mode='rb') as testing_data:
    test = pickle.load(testing_data)

# categorize features and labels
x_test, y_test = test['features'], test['labels']

# convert to grayscale and normalization of the test data
x_test_gray  = to_normalize(x_test)

# check the shape of the test set
x_test_gray.shape

### Load the pre-trained model and get a summary of the architecture

In [ ]:
# load the pre-trained model
new_cnn_model = models.load_model('road-sign_model_version_SDC.h5')

# show architecture
new_cnn_model.summary()

### Check the model accuracy viability using the test set

In [ ]:
# check the accuracy of the loaded model
from sklearn.utils import shuffle
x_test_gray, y_test = shuffle(x_test_gray, y_test)

loss, acc = new_cnn_model.evaluate(x_test_gray, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

## Evaluate the pre-trained model on test set and internet data

In this section we'll test our model performance on internet data and test sets.

### Evaluating the model on the test set

In [ ]:
#Printing the classification report
from sklearn.metrics import classification_report
predictions = np.argmax(new_cnn_model.predict(x_test_gray), axis=-1)
print(classification_report(y_test,predictions))

In [ ]:
# show predictions on test set
img_batch = 90
for i in range(0,12):
    plt.subplot(4,3,i+1)
    plt.imshow(x_test_gray[i+img_batch].squeeze(), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predictions[i+img_batch], y_test[i+img_batch]))
    plt.tight_layout()

### Evaluating the model on real world data

We'll test our model with some real world/internet **data**

In [ ]:
from PIL import Image
from importlib import reload
import utils; reload(utils)
from utils import *

# definition of the internet test directory
new_imgs_dir = "./german_road_signs/"

# get all the images in the folder and apply the preprocessing to the internet images
new_imgs = get_imgs_from_folder(new_imgs_dir)

# convert the images to grayscale and check the grayscale images shape
new_imgs_grayscale = np.asarray(list(map(lambda img: to_grayscale(img), new_imgs)))
new_imgs_grayscale.shape

#### Create new labels corresponding to our imported images

In [ ]:
# Let's also create the labels for our testing images
new_img_lbs = np.array([1,2,35, 29, 32, 18, 9, 9, 27, 12, 12, 25, 11, 14, 14, 14, 26, 8, 15, 12, 16])

# check the grayscale images after converting RGB to grayscale
show_imgs(new_imgs_grayscale, new_img_lbs, "New Traffic Sign Images (Grayscale)", cols=len(new_imgs_grayscale), 
                fig_size=(15,15), show_ticks=False)

In [ ]:
# Convert to a format for Tensorflow inputs
new_imgs_grayscale = np.reshape(new_imgs_grayscale, (new_imgs_grayscale.shape[0], 32, 32, 1))

# check new_imgs_grayscale shape
new_imgs_grayscale.shape

In [ ]:
# check the predictions
predictions = np.argmax(new_cnn_model.predict(new_imgs_grayscale), axis=-1)
predictions

#### Show some predictions 

In [ ]:
# by defaults
number_img = len(new_imgs)
start_number_img = 0
end_number_img = len(new_imgs)/2

# show the first seventh predictions
for i in range(start_number_img,round(end_number_img)):
  plt.subplot(4,3,i+1)
  plt.imshow(new_imgs_grayscale[i].squeeze(), cmap='gray', interpolation='none')
  plt.title("Predicted : {}, \n Class : {}".format(predictions[i], new_img_lbs[i]))
  plt.tight_layout()

In [ ]:
# check the prediction accuracy by summing all the right predictions and divided by the total number of predictions
prediction_accuracy = (np.sum(new_img_lbs == predictions) / len(predictions)) * 100
print("Predictional accuracy on new images: {:5.2f}%".format(prediction_accuracy))